In [ ]:
# hide
# default_exp utils

# Sciflow utils

In [31]:
# export

import os
import subprocess
from pathlib import Path

from turbodbc import connect, make_options, Megabytes
import nbdev
from fastcore.script import call_parse
from nbdev.clean import nbdev_clean_nbs
from nbqa.find_root import find_project_root

In [32]:
%load_ext autoreload
%autoreload 2

# Paths

In [33]:
# export


def lib_path(*lib_relative_path):
    lib_root_path = find_project_root(srcs=(str(Path(".").resolve()),))
    return Path(os.path.join(lib_root_path, *lib_relative_path))

In [34]:
assert str(lib_path("nbs")).endswith("sciflow/nbs")
assert Path("test/test_clustering.ipynb").resolve() == lib_path(
    "nbs", "test", "test_clustering.ipynb"
)

# Sciflow Clean

In [35]:
# export


@call_parse
def sciflow_clean():
    nbdev.clean.cell_metadata_keep = nbdev.clean.cell_metadata_keep + ["tags"]
    nbdev_clean_nbs()

# ODBC Connection

In [73]:
# export


def prepare_env(env_file_path: str = None):
    if env_file_path is None:
        env_file_path = os.path.expanduser('~/.sciflow/env')
    # TODO create this for user
    if not os.path.exists(env_file_path):
        raise EnvironmentError(f'You need to create a Sciflow env file at: {env_file_path}')
    with(open(env_file_path, 'r')) as env_file:
        for line in env_file.readlines():
            key, value = line.strip().split('=', 1)
            os.environ[key.replace('export ', '')] = value
            print(key.replace('export ', ''), value)

In [74]:
# export


def odbc_connect(options = None):
    if options is None:
        options = make_options(
            fetch_wchar_as_char=True,
            autocommit=True,
            read_buffer_size=Megabytes(5),
        )
    connection = connect(driver=os.environ['ODBC_DRIVER'],
                         host=os.environ['ODBC_HOST'],
                         port=os.environ['ODBC_PORT'],
                         uid=os.environ['ODBC_USER'],
                         pwd=os.environ['ODBC_PWD'],
                         SSL=1,
                         TrustedCerts=os.environ['SSL_CERTS'],
                         turbodbc_options=options,
                )
    return connection

In [75]:
# export


def query_odbc(conn, sql, strings_as_dictionary=False, adaptive_integers=False, 
               strings_to_categorical=False, date_as_object=False):
    with conn.cursor() as cursor:
        df = (cursor.execute(sql)
                            .fetchallarrow(
                                strings_as_dictionary=False,
                                adaptive_integers=False,
                            )
                            .to_pandas(
                                strings_to_categorical=False,
                                date_as_object=False,
                            )
                        )
    return df

In [76]:
import turbodbc
conn = odbc_connect()
assert(type(conn) == turbodbc.connection.Connection)
assert(query_odbc(conn, "SELECT 1 AS test_col")['test_col'].iloc[0] == 1)